<a href="https://colab.research.google.com/github/LITHUVARSHNI/Codsoftintern_lithu/blob/main/Movie_genre_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Create a machine learning model that can predict the genre of amovie based on its plot summary or other textual information. Youcan use techniques like TF-IDF or word embeddings with classifierssuch as Naive Bayes, Logistic Regression, or Support VectorMachines.**

IMPORTING THE NECESSARY LIBRARIES

In [5]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk
import string
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report

LOADING THE TRAINING AND TESTING DATA

In [6]:
train_data = pd.read_csv("train_data.txt", sep=':::', names=['Title', 'Genre', 'Description'], engine='python')

In [7]:
test_data = pd.read_csv("test_data.txt", sep=':::', names=['Id', 'Title', 'Description'], engine='python')
test_data.head()

,Id,Title,Description
0,1,Edgar's Lunch (1998),"L.R. Brane loves his life - his car, his apar..."
1,2,La guerra de papá (1977),"Spain, March 1964: Quico is a very naughty ch..."
2,3,Off the Beaten Track (2010),One year in the life of Albin and his family ...
3,4,Meu Amigo Hindu (2015),"His father has died, he hasn't spoken with hi..."
4,5,Er nu zhai (1955),Before he was known internationally as a mart...


'*clean_text*' function is used for preprocessing text data (removing handles, URLs, punctuation, stopwords, and performing stemming)

In [10]:
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer

# Download necessary NLTK data
nltk.download('punkt')
nltk.download('stopwords')

# Initialize Lancaster Stemmer
stemmer = LancasterStemmer()

# Define the clean_text function
def clean_text(text):
    text = text.lower()  # Lowercase all characters
    text = re.sub(r'@\S+', '', text)  # Remove Twitter handles
    text = re.sub(r'http\S+', '', text)  # Remove URLs
    text = re.sub(r'pic.\S+', '', text)
    text = re.sub(r"[^a-zA-Z']", ' ', text)  # Keep only characters and single quotes
    text = re.sub(r'\s+[a-zA-Z]\s+', ' ', text + ' ')  # Keep words with length > 1 only
    text = "".join([i for i in text if i not in string.punctuation])

    # Tokenize the words
    words = nltk.word_tokenize(text)

    # Remove stopwords using nltk stopwords set
    stop_words = set(stopwords.words('english'))
    words = [word for word in words if word.lower() not in stop_words and len(word) > 2]

    # Join the words back into a cleaned text
    text = " ".join(words)

    # Remove repeated/leading/trailing spaces
    text = re.sub("\s[\s]+", " ", text).strip()

    return text

# Assuming train_data and test_data are DataFrames with a 'Description' column
# Apply the clean_text function to the 'Description' column in the training and test data
train_data['Text_cleaning'] = train_data['Description'].apply(clean_text)
test_data['Text_cleaning'] = test_data['Description'].apply(clean_text)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


*'TfidfVectorizer'* fits and transforms the cleaned training text data into TF-IDF features

In [11]:
tfidf_vectorizer = TfidfVectorizer()

# Fit and transform the training data
X_train = tfidf_vectorizer.fit_transform(train_data['Text_cleaning'])

# Transform the test data
X_test = tfidf_vectorizer.transform(test_data['Text_cleaning'])

EVALUATION OF MODEL

In [12]:
# Split the data into training and validation sets
X = X_train
y = train_data['Genre']
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and train a Multinomial Naive Bayes classifier
classifier = MultinomialNB()
classifier.fit(X_train, y_train)

# Make predictions on the validation set
y_pred = classifier.predict(X_val)

# Evaluate the performance of the model
accuracy = accuracy_score(y_val, y_pred)
print("Validation Accuracy:", accuracy)
print(classification_report(y_val, y_pred))

Validation Accuracy: 0.44431041415359873
               precision    recall  f1-score   support

      action        0.00      0.00      0.00       232
       adult        0.00      0.00      0.00        94
   adventure        0.00      0.00      0.00       137
   animation        0.00      0.00      0.00        90
   biography        0.00      0.00      0.00        45
      comedy        0.68      0.04      0.07      1379
       crime        0.00      0.00      0.00       101
 documentary        0.54      0.90      0.67      2409
       drama        0.38      0.89      0.53      2465
      family        0.00      0.00      0.00       138
     fantasy        0.00      0.00      0.00        52
   game-show        0.00      0.00      0.00        38
     history        0.00      0.00      0.00        49
      horror        0.00      0.00      0.00       391
       music        0.00      0.00      0.00       122
     musical        0.00      0.00      0.00        46
     mystery        0.0

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [13]:
import pickle
pickle.dump(tfidf_vectorizer,open('vectorizer.pkl','wb'))
pickle.dump(classifier,open('model.pkl','wb'))